# TreeMethods Review 

## Decision Tree, ....

- How to choose to split in some features first and not in others, for this we will use Entropy H(x) and information gain  IG(S,A)(mathematical methods) to choose the best split

## Random Forest
- we can use many tress with a random rample features chosen  ...... plus details in the future.

- take the average vote of all decision Tree that we made then take a decision at the most frequent 

- The point of Random Forest is pass the problem of one very Strong Feature. because in this case all the decision Tree will use this feature as the top split == > resulting in an Ensemble of similar trees that are highly correlated what does not not significantly reduce variance. For this reason we use random select feature in each tree, as keeping randomly features form each split Random Forest Decorrelates the Trees, such that the averaging process can reduce the variance of the resulting model


## Gradient Boosted Trees 

- involves Tree fondamental elemetns 
    - loss function to be optimized  : a function to determine how far off our prediction are for expamle (Squared Error) for regression and 
    Logarithmic Loss for classification
    - weak learner to make predictions : in Decision trees are used as the weak learner in gradient boosting 
    - an additive model to ass a weak learners to minimze the loss function
        - train a wwak model m using data samples drawn according to some weight distribution
        - increase the weight of samples that are misclassified by the model m, and decrease the weight of the samples are classified correctly in the model m 
        - Train new weak model using the samples drawn according to the updates wight distribution.
        
    

## Start un Example 

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("mytree").getOrCreate()
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
# we can use the for regression as well 

In [4]:
data = spark.read.format("libsvm").load("../../Data/sample_libsvm_data.txt")

In [5]:
data.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [6]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [7]:
train_data ,test_data = data.randomSplit([.7,.3])

In [10]:
dtc = DecisionTreeClassifier(labelCol="label",featuresCol="features")
# there are other parametre we can play with knowing the mathmatical backround 

In [12]:
rfc = RandomForestClassifier(numTrees = 100 ) 
gbt = GBTClassifier()

In [13]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [14]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [15]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[121,122,123...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[122,123,148...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[128,129,130...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[153,154,155...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[154,155,156...|   [0.0,39.0]|  [0.0,1.0]|       1.0|
|  0.0|(69

In [16]:
gbt_preds.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[121,122,123...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[122,123,148...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[124,125,126...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[126,127,128...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[126,127,128...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[126,127,128...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[127,128,129...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[127,128,129...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[128

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [21]:
acc = MulticlassClassificationEvaluator(metricName="accuracy")

In [23]:
acc.evaluate(dtc_preds)

0.9354838709677419

# Feature Important for random forest for example 

In [28]:
#rfc_model.featureImportances 

# the hight number the most important the features is for the models 

# Real Example for  Decision Tree, Random Forest , Gradien Boosting 

In [29]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

In [33]:
data = spark.read.csv("../../Data/College.csv",header=True,inferSchema=True)
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

# we will try to predicted the private feature

In [32]:
from pyspark.ml.feature import VectorAssembler

In [34]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol="features")

In [35]:
output = assembler.transform(data)

In [36]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Private",outputCol="PrivateIndex")

In [39]:
output_fixed = indexer.fit(output).transform(output)

In [40]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [42]:
data_final = output_fixed.select(["features","PrivateIndex"])

In [43]:
train_data, test_data = data_final.randomSplit([.7,.3])

# Creating  the model 

In [44]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

In [47]:
randomForest = RandomForestClassifier (labelCol="PrivateIndex")
GradBoos = GBTClassifier(labelCol="PrivateIndex")
DeciTree = DecisionTreeClassifier(labelCol="PrivateIndex")

In [48]:
randomForest_model = randomForest.fit(train_data)
GradBoos_model = GradBoos.fit(train_data)
DeciTree_model = DeciTree.fit(train_data)

# prediction

In [50]:
randomForestPred = randomForest_model.transform(test_data)
GradBoos_modelPred = GradBoos_model.transform(test_data)
DeciTree_model_modelPred = DeciTree_model.transform(test_data)

In [52]:
randomForestPred.printSchema()
GradBoos_modelPred.printSchema()
DeciTree_model_modelPred.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



# Evaluation Matrix 

In [53]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [54]:
my_binary_eval = BinaryClassificationEvaluator(labelCol="PrivateIndex")

In [55]:
resutl = my_binary_eval.evaluate(randomForestPred)

In [58]:
print("AUC for Random Forest")
resutl

AUC for Random Forest


0.9711493933500035

In [59]:
resutl = my_binary_eval.evaluate(DeciTree_model_modelPred)
print("AUC for Decision Tree")
resutl

AUC for Decision Tree


0.899370195424655

In [60]:
resutl = my_binary_eval.evaluate(GradBoos_modelPred)
print("AUC for Gradien Boosting ")
resutl

AUC for Gradien Boosting 


0.9574881911642124

# Accuracy,....

In [62]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [65]:
my_multi_eval = MulticlassClassificationEvaluator(labelCol="PrivateIndex", metricName="accuracy")
resutl = my_multi_eval.evaluate(DeciTree_model_modelPred)
print("Accuracy for Decision Tree")
resutl

Accuracy for Decision Tree


0.9243697478991597